## **Pauta C4a**

Cargamos los datos

In [ ]:
from google.colab import files
data = files.upload()

Saving data.zip to data (3).zip


In [ ]:
!unzip data

Archive:  data.zip
replace data/Answer.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/Answer.csv         
  inflating: data/Question.csv       
  inflating: data/salud_mental_tech.db  


# Misión 1

In [ ]:
import csv
import sqlite3
import pandas as pd

In [ ]:
connection = sqlite3.connect('/content/data/salud_mental_tech.db')
cursor = connection.cursor()

In [ ]:
def list_tables(cursor):
  sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
  cursor.execute(sql_query)
  tables = cursor.fetchall()

  for t in tables:
    t_name = t[0]
    sql_query = f"SELECT * FROM {t_name};"
    cursor.execute(sql_query)
    c_names = [description[0] for description in cursor.description]
    print(t_name)
    print(c_names, type(c_names[0]))
    print("\n=========\n")

In [ ]:
list_tables(cursor)

Survey
['SurveyID', 'Description'] <class 'str'>




La función imprime las tablas de la base de datos. Para esto, realiza una consulta sql que entrega el nombre de cada tabla. Luego, se crea una lista con los nombres para iterar sobre esta y se ejecuta otra consulta sql que devuelve las columnas (atributos) de cada tabla, para finalmente imprimirlos.

In [ ]:
def get_table_contents(cursor, t_name, limit=0):
  if limit:
    sql_query = f"SELECT * FROM {t_name} LIMIT {limit};"
  else:
    sql_query = f"SELECT * FROM {t_name}"

  print(t_name)
  cursor.execute(sql_query)
  c_names = [description[0] for description in cursor.description]
  print(c_names)

  for r in cursor.fetchall():
    print(r)
  print("\n=========\n")

In [ ]:
get_table_contents(cursor, "Survey", limit=2)

Survey
['SurveyID', 'Description']
(2014, 'mental health survey for 2014')
(2016, 'mental health survey for 2016')




Esta función recibe un cursor, el nombre de la tabla, y un límite, e imprime los datos de todas las columnas de la tabla. Si no se especifica un límite, imprime todas las filas, de lo contrario, imprime la cantidad de filas determinada (si el límite es mayor a la cantidad de filas, solo imprime las filas existentes). Para realizar esto, se define una consulta SQL (dependiendo de si tiene o no un argumento limit), y luego imprime el nombre de la tabla, el nombre de sus atributos, y las filas correspondientes.

# Misión 2

Con Pandas

In [ ]:
question = pd.read_csv("/content/data/Question.csv")
question.head()

,questiontext,questionid
0,What is your age?,1
1,What is your gender?,2
2,What country do you live in?,3
3,"If you live in the United States, which state ...",4
4,Are you self-employed?,5


Creando la tabla en la base de datos.

In [ ]:
sql_query = "CREATE TABLE Question(questionid INTEGER, questiontext TEXT, PRIMARY KEY(questionid))"
cursor.execute(sql_query)
list_tables(cursor)

Survey
['SurveyID', 'Description'] <class 'str'>


Question
['questionid', 'questiontext'] <class 'str'>




Insertando los datos.

In [ ]:
filas = len(question)
for i in range(filas):
  questionid = int(question.loc[i]["questionid"])
  questiontext = question.loc[i]["questiontext"]
  cursor.execute("INSERT INTO Question(questionid, questiontext) VALUES (?,?)", (questionid, questiontext))

In [ ]:
get_table_contents(cursor, "Question", limit=5)

Question
['questionid', 'questiontext']
(1, 'What is your age?')
(2, 'What is your gender?')
(3, 'What country do you live in?')
(4, 'If you live in the United States, which state or territory do you live in?')
(5, 'Are you self-employed?')




Usando la librería csv

In [ ]:
def import_from_csv(connection, cursor, t_name, csv_path):
  data = []
  
  with open(csv_path, 'r') as f:
    dr = csv.DictReader(f)
    # to_db = [(i['col1'], i['col2']) for i in dr]
    columns = tuple(dr.fieldnames)
    print(columns)
    data = [tuple([row[i] for i in columns]) for row in dr]

    cursor.executemany(f"INSERT INTO {t_name} {columns} VALUES (?, ?);", data)
    connection.commit()

In [ ]:
cursor.execute("DROP TABLE Question")
cursor.execute("CREATE TABLE Question(questionid INTEGER, questiontext TEXT, PRIMARY KEY(questionid))")

In [ ]:
import_from_csv(connection, cursor, "Question", "/content/data/Question.csv")

('questiontext', 'questionid')


In [ ]:
get_table_contents(cursor, "Question", limit=5)

Question
['questionid', 'questiontext']
(1, 'What is your age?')
(2, 'What is your gender?')
(3, 'What country do you live in?')
(4, 'If you live in the United States, which state or territory do you live in?')
(5, 'Are you self-employed?')




# Misión 3

Con pandas

In [ ]:
answer = pd.read_csv("/content/data/Answer.csv")
answer.head()

,AnswerText,SurveyID,UserID,QuestionID
0,37,2014,1,1
1,44,2014,2,1
2,32,2014,3,1
3,31,2014,4,1
4,31,2014,5,1


In [ ]:
sql_query = '''CREATE TABLE Answer
               (SurveyID INTEGER,
                QuestionID INTEGER,
                UserID INTEGER,
                AnswerText TEXT,
                FOREIGN KEY(SurveyID) REFERENCES Survey(SurveyID),
                FOREIGN KEY(QuestionID) REFERENCES Question(questionid)
                PRIMARY KEY(SurveyID, QuestionID, UserID))'''
cursor.execute(sql_query)
list_tables(cursor)

Survey
['SurveyID', 'Description'] <class 'str'>


Question
['questionid', 'questiontext'] <class 'str'>


Answer
['SurveyID', 'QuestionID', 'UserID', 'AnswerText'] <class 'str'>




In [ ]:
filas = len(answer)
for i in range(filas):
  surveyid = int(answer.loc[i]["SurveyID"])
  questionid = int(answer.loc[i]["QuestionID"])
  userid = int(answer.loc[i]["UserID"])
  answertext = answer.loc[i]["AnswerText"]
  cursor.execute("INSERT INTO Answer(SurveyID, QuestionID, UserID, AnswerText) VALUES (?,?,?,?)", (surveyid, questionid, userid, answertext))

In [ ]:
get_table_contents(cursor, "Answer", limit=5)

Answer
['SurveyID', 'QuestionID', 'UserID', 'AnswerText']
(2014, 1, 1, '37')
(2014, 1, 2, '44')
(2014, 1, 3, '32')
(2014, 1, 4, '31')
(2014, 1, 5, '31')




Usando la libreria csv

In [ ]:
cursor.execute("DROP TABLE Answer")
sql_query = '''CREATE TABLE Answer
               (SurveyID INTEGER,
                QuestionID INTEGER,
                UserID INTEGER,
                AnswerText TEXT,
                FOREIGN KEY(SurveyID) REFERENCES Survey(SurveyID),
                FOREIGN KEY(QuestionID) REFERENCES Question(questionid)
                PRIMARY KEY(SurveyID, QuestionID, UserID, AnswerText))'''
cursor.execute(sql_query)

In [ ]:
def import_from_csv(connection, cursor, t_name, csv_path):
  data = []
  
  with open(csv_path, 'r') as f:
    dr = csv.DictReader(f)
    # to_db = [(i['col1'], i['col2']) for i in dr]
    columns = tuple(dr.fieldnames)
    print(columns)
    data = [tuple([row[i] for i in columns]) for row in dr]

    cursor.executemany(f"INSERT INTO {t_name} {columns} VALUES (?, ?, ?, ?);", data)
    connection.commit()

In [ ]:
import_from_csv(connection, cursor, "Answer", "/content/data/Answer.csv")

('AnswerText', 'SurveyID', 'UserID', 'QuestionID')


In [ ]:
get_table_contents(cursor, "Answer", limit=5)

Answer
['SurveyID', 'QuestionID', 'UserID', 'AnswerText']
(2014, 1, 1, '37')
(2014, 1, 2, '44')
(2014, 1, 3, '32')
(2014, 1, 4, '31')
(2014, 1, 5, '31')




In [ ]:
list_tables(cursor)

Survey
['SurveyID', 'Description'] <class 'str'>


Question
['questionid', 'questiontext'] <class 'str'>


Answer
['SurveyID', 'QuestionID', 'UserID', 'AnswerText'] <class 'str'>


